https://docs.juliacn.com/latest/manual/metaprogramming/#关于表达式的函数<br>
已经见过返回 Expr 对象的函数例子：parse 函数，它接受字符串形式的 Julia 代码并返回相应的 Expr。函数也可以接受一个或多个 Expr 对象作为参数，并返回另一个 Expr。这是个简单、提神的例子：

In [6]:
function math_expr(op, op1, op2)
    expr = Expr(:call, op, op1, op2)
    return expr
end
math_expr(:+, 1, Expr(:call, :*, 4, 5))

:(1 + 4 * 5)

作为另一个例子，这个函数将数值参数加倍，但不处理表达式：

In [7]:
function make_expr2(op, opr1, opr2)
    opr1f, opr2f = map(x -> isa(x, Number) ? 2*x : x, (opr1, opr2))
    retexpr = Expr(:call, op, opr1f, opr2f)
    return retexpr
end
make_expr2(:+, 1, 2) |> display
make_expr2(:+, 1, Expr(:call, :*, 5, 8)) |> display

:(2 + 4)

:(2 + 5 * 8)

### https://docs.juliacn.com/latest/manual/metaprogramming/#代码生成

当需要大量重复的样板代码时，为了避免冗余，通常以编程方式生成它。在大多数语言中，这需要一个额外的构建步骤以及生成重复代码的独立程序。在 Julia 中，**表达式插值和 `eval` 允许在通常的程序执行过程中生成这些代码**。例如，考虑下列自定义类型

In [1]:
struct MyNumber
    x::Float64
end

我们想为该类型添加一些方法。在下面的循环中，我们以编程的方式完成此工作：

In [2]:
for op = (:sin, :cos, :tan, :log, :exp)  
    # 并没把for循环写成函数, 但这个和函数处理表达式类似,
    # 所以就放在 此ipynb文件中啦
    eval(quote
        Base.$op(a::MyNumber) = MyNumber($op(a.x))
        # 如, 对 MyNumber类型的数据a 求 sin, 结果就是 a.x的sin
        # 此步可选：结果 也弄成 MyNumber类型(实例化),这叫数域的封闭性, 如整数相加还是整数
    end)
end

In [3]:
x = MyNumber(π)

MyNumber(3.141592653589793)

In [6]:
sin(x) |> display
cos(x) |> display

MyNumber(1.2246467991473532e-16)

MyNumber(-1.0)

在这种方法中，Julia 充当了自己的预处理器，并且允许从语言内部生成代码。使用 `:` 前缀的引用形式编写上述代码会使其更简洁：
```julia
for op = (:sin, :cos, :tan, :log, :exp)
    eval(:(Base.$op(a::MyNumber) = MyNumber($op(a.x))))
end
```
不管怎样，这种使用 `eval(quote(...))` 模式生成语言内部的代码很常见，为此，Julia 自带了一个 **宏`@eval`** 来缩写该模式：
```julia
for op = (:sin, :cos, :tan, :log, :exp)
    @eval Base.$op(a::MyNumber) = MyNumber($op(a.x))
end
```
`@eval` 重写此调用，使其与上面的较长版本完全等价。为了生成较长的代码块，可以把一个代码块作为表达式参数传给 `@eval`：
```julia
@eval begin
    # multiple lines
end
```

```julia
macro eval(ex)
    :(Core.eval($__module__, $(Expr(:quote,ex))))
end
```

In [7]:
for op = (:sin, :cos, :tan, :log, :exp)
    # @eval Base.$op(a::MyNumber) = MyNumber($op(a.x)) # 最好这样 数域封闭
    @eval Base.$op(a::MyNumber) = $op(a.x)
end

In [8]:
x = MyNumber(π)

MyNumber(3.141592653589793)

In [9]:
sin(x) |> display
cos(x) |> display

1.2246467991473532e-16

-1.0

In [11]:
expr = :(1+2)

display(@eval 1+2)

(@eval expr) |> display

eval(QuoteNode(expr)) |> display

3

:(1 + 2)

:(1 + 2)